<h1 align=center>Capstone Project</h1>

<h2>Data Gathering</h2>

In [ ]:
#Importing packages
import numpy as np # library to handle data in a vectorized manner

import pandas as pd
import json # library to handle JSON files

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

In [48]:
#Enter Foursquare credentials
CLIENT_ID = 'KLGUJ4KR5J331FPXXPB1ONE5AHN5LTOYQKKXHMUGK2ID5BEE' # your Foursquare ID
CLIENT_SECRET = 'EDQKNPOHO0YM30SQWTYFLBUNPYEPC2HNDATCY2BVCR4E5FK0' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KLGUJ4KR5J331FPXXPB1ONE5AHN5LTOYQKKXHMUGK2ID5BEE
CLIENT_SECRET:EDQKNPOHO0YM30SQWTYFLBUNPYEPC2HNDATCY2BVCR4E5FK0


In [49]:
#install beautifulsoup for webscraping
!pip install beautifulsoup4

In [50]:
#Gathering data from the website and storing in th bangalore.csv file
from bs4 import BeautifulSoup
url="https://www.indiatvnews.com/pincode/karnataka/bangalore/bangalore-city"
html = requests.get(url).content
df_list = pd.read_html(html,flavor='bs4')
df=df_list[1]
df.to_csv('bangalore.csv')
df.head()

,Office,Taluk,District,State,Pincode
0,A F Station Yelahanka,Bangalore North,Bangalore,KARNATAKA,560063
1,Agram,Bangalore South,Bangalore,KARNATAKA,560007
2,Air Force Hospital,Bangalore North,Bangalore,KARNATAKA,560007
3,Amruthahalli,Bangalore North,Bangalore,KARNATAKA,560092
4,Anandnagar Bangalore,Bangalore North,Bangalore,KARNATAKA,560024


In [51]:
#Method to get coordinates of the localities 
def coords(off,dis):
    address=off+','+dis
    geolocator = Nominatim(user_agent="ny_explorer")
    try:
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
    except:
        return(np.nan,np.nan)
    return(latitude, longitude)

In [52]:
#Adding the Latitude and Longitude columns to the dataframe
long=list()
lat=list()
for office,district in zip(df['Office'],df['District']):
    la,lo=coords(office,district)
    lat.append(la)
    long.append(lo)
df['Latitude']=lat
df['Longitude']=long
df.head()

,Office,Taluk,District,State,Pincode,Latitude,Longitude
0,A F Station Yelahanka,Bangalore North,Bangalore,KARNATAKA,560063,NaN,NaN
1,Agram,Bangalore South,Bangalore,KARNATAKA,560007,NaN,NaN
2,Air Force Hospital,Bangalore North,Bangalore,KARNATAKA,560007,12.964027,77.627500
3,Amruthahalli,Bangalore North,Bangalore,KARNATAKA,560092,13.066513,77.596624
4,Anandnagar Bangalore,Bangalore North,Bangalore,KARNATAKA,560024,13.033377,77.589523


<h2>Data Cleaning</h2>

In [53]:
#To check duplicate datas and null values
df.Latitude.value_counts()

12.979120    3
13.046453    2
13.335238    2
13.016341    2
12.877349    2
12.800444    2
12.971889    2
13.133436    2
12.929273    2
13.000509    2
12.897971    1
13.089139    1
12.968444    1
13.053333    1
12.915221    1
12.928367    1
13.215192    1
12.985852    1
12.886018    1
12.912192    1
12.742629    1
13.033117    1
12.998577    1
13.000405    1
12.979745    1
12.689957    1
13.133187    1
13.047950    1
12.910639    1
12.754299    1
12.968003    1
13.031462    1
12.906985    1
12.877710    1
12.935146    1
13.027331    1
12.956997    1
12.895583    1
13.033019    1
12.936265    1
12.945401    1
13.010868    1
12.962321    1
13.030013    1
13.056649    1
12.950624    1
12.856532    1
12.969637    1
13.204259    1
12.971777    1
12.961274    1
12.958577    1
13.171580    1
12.977673    1
12.964027    1
13.025800    1
12.858081    1
12.915113    1
12.954345    1
13.176735    1
12.976395    1
12.818896    1
13.107470    1
13.012022    1
13.002655    1
13.427934    1
12.990879 

In [54]:
#Removing null values
df_new= df[pd.notnull(df['Latitude'])]
df_new

,Office,Taluk,District,State,Pincode,Latitude,Longitude
2,Air Force Hospital,Bangalore North,Bangalore,KARNATAKA,560007,12.964027,77.627500
3,Amruthahalli,Bangalore North,Bangalore,KARNATAKA,560092,13.066513,77.596624
4,Anandnagar Bangalore,Bangalore North,Bangalore,KARNATAKA,560024,13.033377,77.589523
5,Arabic College,Bangalore North,Bangalore,KARNATAKA,560045,13.030013,77.620872
6,Attur,Bangalore North,Bangalore,KARNATAKA,560064,13.107470,77.565685
7,Austin Town,Bangalore North,Bangalore,KARNATAKA,560047,12.961274,77.615294
8,Banaswadi,Bangalore North,Bangalore,KARNATAKA,560043,13.014162,77.651854
10,Bangalore International Airport,NaN,Bangalore,KARNATAKA,560300,13.204259,77.706567
12,Bellandur,Bangalore South,Bangalore,KARNATAKA,560103,12.979120,77.591300
13,Benson Town,Bangalore North,Bangalore,KARNATAKA,560046,12.997803,77.604175


In [55]:
# drop unnecessary columns
column_to_drop = ['Taluk','State']
df_new.drop(columns=column_to_drop, axis=1,inplace=True)
df_new

,Office,District,Pincode,Latitude,Longitude
2,Air Force Hospital,Bangalore,560007,12.964027,77.627500
3,Amruthahalli,Bangalore,560092,13.066513,77.596624
4,Anandnagar Bangalore,Bangalore,560024,13.033377,77.589523
5,Arabic College,Bangalore,560045,13.030013,77.620872
6,Attur,Bangalore,560064,13.107470,77.565685
7,Austin Town,Bangalore,560047,12.961274,77.615294
8,Banaswadi,Bangalore,560043,13.014162,77.651854
10,Bangalore International Airport,Bangalore,560300,13.204259,77.706567
12,Bellandur,Bangalore,560103,12.979120,77.591300
13,Benson Town,Bangalore,560046,12.997803,77.604175


In [56]:
#resetting the index
df_new.reset_index(inplace=True,drop=True)
df_new.head()

,Office,District,Pincode,Latitude,Longitude
0,Air Force Hospital,Bangalore,560007,12.964027,77.627500
1,Amruthahalli,Bangalore,560092,13.066513,77.596624
2,Anandnagar Bangalore,Bangalore,560024,13.033377,77.589523
3,Arabic College,Bangalore,560045,13.030013,77.620872
4,Attur,Bangalore,560064,13.107470,77.565685


In [57]:
# for combining neighbourhood of a single postal code(location) i.e removing duplicate datas and combining into one
l=list()
for a in df_new['Latitude'].unique():
    ne=""
    df_post = df_new[df_new['Latitude']==a].reset_index(drop=True)
    neighs = df_post.loc[:,'Office']
    for n in neighs:
        ne=ne+','+n
    df_post1 = df_post.iloc[0]
    df_post1['Office']=ne[1:]
    l.append(df_post1)
dff = pd.DataFrame(l)
dff.reset_index(drop=True,inplace=True)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [58]:
#Renaming the column 'Office' as 'Locality' which is much suited
dff.columns = ['Locality']+list(map(lambda x: x , dff.columns[1:]))
dff.columns

Index(['Locality', 'District', 'Pincode', 'Latitude', 'Longitude'], dtype='object')

In [59]:
#dff.to_csv('Bangalore_cleaned.csv')

In [60]:
#Visualizing the data points on the map
#from this we realise locations far from the bangalore centre are also present which needs to be removed
latitude=12.9180502
longitude=77.5922633618582
map_bang = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, district, location in zip(dff['Latitude'], dff['Longitude'], dff['District'], dff['Locality']):
    label = '{},{}'.format(location,district)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bang)  
    
map_bang

In [61]:
#installing haversine which is used to calculate distance between the coords
!pip install haversine

In [62]:
#removing areas which are 35 kms away from bangalore center so as to make the data more concentrated towards core bangalore.
from haversine import haversine, Unit
bang = coords('bangalore','bangalore')
l=list()
for lat,long in zip(dff['Latitude'], dff['Longitude']):
    if haversine((lat,long),bang)<35:
        df_post1 = dff[dff['Latitude']==lat]
        l.append(df_post1.iloc[0])
new_df=pd.DataFrame(l)
new_df.shape

(142, 5)

In [63]:
new_df.head()

,Locality,District,Pincode,Latitude,Longitude
0,Air Force Hospital,Bangalore,560007,12.964027,77.627500
1,Amruthahalli,Bangalore,560092,13.066513,77.596624
2,Anandnagar Bangalore,Bangalore,560024,13.033377,77.589523
3,Arabic College,Bangalore,560045,13.030013,77.620872
4,Attur,Bangalore,560064,13.107470,77.565685


In [64]:
#Storing final cleaned and required data
new_df.to_csv('Bangalore_cleaned.csv')

In [65]:
#To visualize the final data points
latitude=12.9180502
longitude=77.5922633618582
map_bang = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, district, location in zip(new_df['Latitude'], new_df['Longitude'], new_df['District'], new_df['Locality']):
    label = '{},{}'.format(location,district)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bang)  
    
map_bang

<h2>Gathering more data about neighbourhood using Foursquare Api </h2>

In [66]:
#A method To get venues near every location considering 500 metres radius and maximum of 100 venues per location
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [67]:
#Storing every venues with the locality
bangalore_venues = getNearbyVenues(names=new_df['Locality'],
                                   latitudes=new_df['Latitude'],
                                   longitudes=new_df['Longitude']
                                  )

Air Force Hospital
Amruthahalli
Anandnagar Bangalore
Arabic College
Attur
Austin Town
Banaswadi
Bangalore International Airport
Bellandur,Ashoknagar Bangalore,Bangalore City
Benson Town
Bhattarahalli
Byatarayanapura
C.V.Raman Nagar
CMP Centre & School
Devasandra
Doddanekkundi
Domlur
Dr. Ambedkar Veedhi
EPIP
Hoodi
Horamavu
Indiranagar Bangalore
Jakkur
Kadugodi
Kannamangala
Kothanur
Krishnarajapuram R S,Krishnarajapuram
Kundalahalli
Lingarajapuram
Mahadevapura
Mahatma Gandhi Road
Maruthi Sevanagar
Medimallasandra
Museum Road
NAL
New Thippasandra
Panathur
R T Nagar
Rajanakunte
Ramamurthy Nagar
Richmond Town
Sadashivanagar
Singanayakanahalli
Venkateshapura
Vidhana Soudha
Vimanapura
Virgonagar
Viveknagar Bangalore
Whitefield
Yelahanka
Yelahanka Satellite Town
Adugodi
Anjanapura
Banashankari III Stage
Banashankari
Bangalore Corporation Building
Bannerghatta Road
Bannerghatta
Basavanagudi
Begur
Bolare
Bommasandra Industrial Estate
Chamrajpet Bangalore
Chandapura,Chandapura
Chickpet
Chikkalasa

In [68]:
#total no of venues found
print(bangalore_venues.shape)
bangalore_venues.tail()

(1032, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
1027,Kannur,13.103799,77.601617,"Udupi Grand, Kogilu Cross",13.103455,77.600627,Fast Food Restaurant
1028,Kannur,13.103799,77.601617,Yelahanka lake,13.103729,77.602205,Lake
1029,Kannur,13.103799,77.601617,Gobi Adda,13.103732,77.602401,Food Truck
1030,Madanayakanahalli,13.059247,77.461511,Khimaj Caterers,13.060353,77.460115,Restaurant
1031,Neriga,12.911766,77.776525,Shristhi Village,12.913261,77.773636,Resort


<h2>Data Analysis Phase</h2>

In [69]:
bangalore_venues.groupby('Neighborhood').count()
#bangalore_venues[['Venue', 'Venue Category']].corr() 

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adugodi,5,5,5,5,5,5
Air Force Hospital,2,2,2,2,2,2
Amruthahalli,2,2,2,2,2,2
Anandnagar Bangalore,6,6,6,6,6,6
Anekal,2,2,2,2,2,2
Anjanapura,1,1,1,1,1,1
Arabic College,1,1,1,1,1,1
Austin Town,2,2,2,2,2,2
Bagalgunte,4,4,4,4,4,4


In [100]:
bangalore_venues.to_csv('Bangalore_locations_with_neighbourhood.csv')

In [70]:
#trial = bangalore_venues[bangalore_venues['Venue Category']=='Café']
#trial

In [71]:
#latitude=12.9180502
#longitude=77.5922633618582
#map_bang = folium.Map(location=[latitude, longitude], zoom_start=10)

## add markers to map
#for lat, lng, district, location in zip(trial['Venue Latitude'], trial['Venue Longitude'], trial['Venue'], trial['Neighborhood']):
#    label = '{},{}'.format(district,location)
#    label = folium.Popup(label, parse_html=True)
#    folium.CircleMarker(
#        [lat, lng],
#        radius=5,
#        popup=label,
#        color='blue',
#        fill=True,
#        fill_color='#3186cc',
#        fill_opacity=0.7,
#        parse_html=False).add_to(map_bang)  
#    
#map_bang

In [72]:
print('There are {} uniques categories.'.format(len(bangalore_venues['Venue Category'].unique())))

There are 183 uniques categories.


<h2>Analyze each neighbourhood</h2>

In [73]:
# one hot encoding
bangalore_onehot = pd.get_dummies(bangalore_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bangalore_onehot['Neighborhood'] = bangalore_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [bangalore_onehot.columns[-1]] + list(bangalore_onehot.columns[:-1])
bangalore_onehot = bangalore_onehot[fixed_columns]

bangalore_onehot.head()

,Neighborhood,ATM,Accessories Store,Airport Gate,Airport Lounge,American Restaurant,Andhra Restaurant,Antique Shop,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Badminton Court,Bakery,Bar,Basketball Court,Bengali Restaurant,Bike Shop,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Camera Store,Candy Store,Cantonese Restaurant,Capitol Building,Casino,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dhaba,Diner,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Health Food Store,Historic Site,Hockey Arena,Hookah Bar,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kebab Restaurant,Kerala Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Memorial Site,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Optical Shop,Other Nightlife,Other Repair Shop,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Rajasthani Restaurant,Resort,Restaurant,Road,Salad Place,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soccer Field,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Sports Bar,Sports Club,Steakhouse,Street Food Gathering,Supermarket,Swiss Restaurant,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Toll Plaza,Toy / Game Store,Trail,Train Station,Travel & Transport,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Vineyard,Wine Shop,Women's Store,Yoga Studio
0,Air Force Hospital,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Air Force Hospital,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Amruthahalli,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Amruthahalli,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [74]:
bangalore_grouped = bangalore_onehot.groupby('Neighborhood').mean().reset_index()
bangalore_grouped

,Neighborhood,ATM,Accessories Store,Airport Gate,Airport Lounge,American Restaurant,Andhra Restaurant,Antique Shop,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Badminton Court,Bakery,Bar,Basketball Court,Bengali Restaurant,Bike Shop,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Camera Store,Candy Store,Cantonese Restaurant,Capitol Building,Casino,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dhaba,Diner,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Health Food Store,Historic Site,Hockey Arena,Hookah Bar,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kebab Restaurant,Kerala Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Memorial Site,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Optical Shop,Other Nightlife,Other Repair Shop,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Rajasthani Restaurant,Resort,Restaurant,Road,Salad Place,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soccer Field,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Sports Bar,Sports Club,Steakhouse,Street Food Gathering,Supermarket,Swiss Restaurant,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Toll Plaza,Toy / Game Store,Trail,Train Station,Travel & Transport,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Vineyard,Wine Shop,Women's Store,Yoga Studio
0,Adugodi,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.200000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.200000,0.00,0.000000,0.000000,0.200000,0.0,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.00,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0

In [75]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [76]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = bangalore_grouped['Neighborhood']

for ind in np.arange(bangalore_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bangalore_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Adugodi,Ice Cream Shop,Bus Station,Playground,Café,Athletics & Sports
1,Air Force Hospital,Casino,Multiplex,Yoga Studio,Eastern European Restaurant,Food & Drink Shop
2,Amruthahalli,Convenience Store,Shoe Store,Eastern European Restaurant,Food & Drink Shop,Food
3,Anandnagar Bangalore,Pharmacy,Grocery Store,Motorcycle Shop,Garden Center,Hotel
4,Anekal,ATM,Camera Store,Food & Drink Shop,Food,Flower Shop


In [77]:
# set number of clusters
kclusters = 5

bangalore_grouped_clustering = bangalore_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bangalore_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 1, 3, 0, 0], dtype=int32)

In [78]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

bangalore_merged = new_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
bangalore_merged = bangalore_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Locality')

bangalore_merged.tail() # check the last columns!

,Locality,District,Pincode,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
146,Kannur,Bangalore,562149,13.103799,77.601617,0.0,Food Truck,Vegetarian / Vegan Restaurant,Fast Food Restaurant,Lake,Eastern European Restaurant
147,Madanayakanahalli,Bangalore,562162,13.059247,77.461511,2.0,Restaurant,Yoga Studio,Dry Cleaner,Food & Drink Shop,Food
148,Mayasandra,Bangalore,562107,12.877244,77.377949,NaN,NaN,NaN,NaN,NaN,NaN
149,Neriga,Bangalore,562125,12.911766,77.776525,4.0,Resort,Yoga Studio,Dry Cleaner,Food & Drink Shop,Food
152,Yadavanahalli,Bangalore,562107,12.789855,77.751454,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
bangalore_merged.tail() 

,Locality,District,Pincode,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
146,Kannur,Bangalore,562149,13.103799,77.601617,0.0,Food Truck,Vegetarian / Vegan Restaurant,Fast Food Restaurant,Lake,Eastern European Restaurant
147,Madanayakanahalli,Bangalore,562162,13.059247,77.461511,2.0,Restaurant,Yoga Studio,Dry Cleaner,Food & Drink Shop,Food
148,Mayasandra,Bangalore,562107,12.877244,77.377949,NaN,NaN,NaN,NaN,NaN,NaN
149,Neriga,Bangalore,562125,12.911766,77.776525,4.0,Resort,Yoga Studio,Dry Cleaner,Food & Drink Shop,Food
152,Yadavanahalli,Bangalore,562107,12.789855,77.751454,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
bangalore_merged.dropna(inplace=True)
bangalore_merged.reset_index(drop=True,inplace=True)
bangalore_merged.tail()

,Locality,District,Pincode,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
114,Doddajala,Bangalore,562157,13.176735,77.652050,0.0,Mountain,Yoga Studio,Eastern European Restaurant,Food & Drink Shop,Food
115,Hunasamaranahalli,Bangalore,562157,13.144010,77.619124,3.0,Indian Restaurant,Coffee Shop,Lake,Food & Drink Shop,Food
116,Kannur,Bangalore,562149,13.103799,77.601617,0.0,Food Truck,Vegetarian / Vegan Restaurant,Fast Food Restaurant,Lake,Eastern European Restaurant
117,Madanayakanahalli,Bangalore,562162,13.059247,77.461511,2.0,Restaurant,Yoga Studio,Dry Cleaner,Food & Drink Shop,Food
118,Neriga,Bangalore,562125,12.911766,77.776525,4.0,Resort,Yoga Studio,Dry Cleaner,Food & Drink Shop,Food


In [88]:
bangalore_merged['Cluster Labels'].unique()

array([0., 1., 3., 2., 4.])

In [89]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bangalore_merged['Latitude'], bangalore_merged['Longitude'], bangalore_merged['Locality'],bangalore_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [95]:
bangalore_merged.loc[bangalore_merged['Cluster Labels'] == 0, bangalore_merged.columns[[0] + list(range(5, bangalore_merged.shape[1]))]]

,Locality,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Air Force Hospital,0.0,Casino,Multiplex,Yoga Studio,Eastern European Restaurant,Food & Drink Shop
1,Amruthahalli,0.0,Convenience Store,Shoe Store,Eastern European Restaurant,Food & Drink Shop,Food
2,Anandnagar Bangalore,0.0,Pharmacy,Grocery Store,Motorcycle Shop,Garden Center,Hotel
5,Banaswadi,0.0,Vegetarian / Vegan Restaurant,Kerala Restaurant,Grocery Store,Pharmacy,BBQ Joint
6,Bangalore International Airport,0.0,Airport Lounge,Coffee Shop,Airport Gate,Vegetarian / Vegan Restaurant,Indian Restaurant
7,"Bellandur,Ashoknagar Bangalore,Bangalore City",0.0,Hotel,Vineyard,Capitol Building,Park,Cosmetics Shop
8,Benson Town,0.0,Pakistani Restaurant,Halal Restaurant,Bakery,Café,Yoga Studio
9,Bhattarahalli,0.0,Ice Cream Shop,Café,Hotel,Airport Gate,Airport Lounge
12,CMP Centre & School,0.0,Soccer Field,Asian Restaurant,Hotel,Café,Business Service
13,Doddanekkundi,0.0,Coffee Shop,Kerala Restaurant,Clothing Store,Multiplex,Movie Theater


In [96]:
bangalore_merged.loc[bangalore_merged['Cluster Labels'] == 1, bangalore_merged.columns[[0] + list(range(5, bangalore_merged.shape[1]))]]

,Locality,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
3,Arabic College,1.0,Indian Restaurant,Yoga Studio,Dry Cleaner,Food & Drink Shop,Food
38,Singanayakanahalli,1.0,Indian Restaurant,Café,Yoga Studio,Eastern European Restaurant,Food & Drink Shop
86,Thalaghattapura,1.0,Indian Restaurant,Yoga Studio,Dry Cleaner,Food & Drink Shop,Food
104,Science Institute,1.0,Indian Restaurant,Yoga Studio,Dry Cleaner,Food & Drink Shop,Food


In [97]:
bangalore_merged.loc[bangalore_merged['Cluster Labels'] == 2, bangalore_merged.columns[[0] + list(range(5, bangalore_merged.shape[1]))]]

,Locality,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
45,Yelahanka,2.0,Restaurant,Motorcycle Shop,Train Station,Dry Cleaner,Food
58,"Chandapura,Chandapura",2.0,Restaurant,Asian Restaurant,Dry Cleaner,Food & Drink Shop,Food
95,Magadi Road,2.0,ATM,Restaurant,Dry Cleaner,Food & Drink Shop,Food
117,Madanayakanahalli,2.0,Restaurant,Yoga Studio,Dry Cleaner,Food & Drink Shop,Food


In [98]:
bangalore_merged.loc[bangalore_merged['Cluster Labels'] == 3, bangalore_merged.columns[[0] + list(range(5, bangalore_merged.shape[1]))]]

,Locality,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
4,Austin Town,3.0,Indian Restaurant,Italian Restaurant,Yoga Studio,Dry Cleaner,Food & Drink Shop
10,Byatarayanapura,3.0,Indian Restaurant,Wine Shop,Burger Joint,Andhra Restaurant,Flower Shop
11,C.V.Raman Nagar,3.0,Indian Restaurant,Pizza Place,Park,Shop & Service,Department Store
15,Dr. Ambedkar Veedhi,3.0,Hotel,Indian Restaurant,Capitol Building,Sandwich Place,Dry Cleaner
17,Hoodi,3.0,Indian Restaurant,Yoga Studio,Bus Station,Eastern European Restaurant,Food & Drink Shop
18,Horamavu,3.0,Indian Restaurant,Chinese Restaurant,Athletics & Sports,Dry Cleaner,Yoga Studio
30,NAL,3.0,Indian Restaurant,Chinese Restaurant,Kerala Restaurant,Dessert Shop,Café
31,New Thippasandra,3.0,Indian Restaurant,Grocery Store,Park,Food Truck,Fast Food Restaurant
33,R T Nagar,3.0,Indian Restaurant,Ice Cream Shop,Bakery,Snack Place,Vegetarian / Vegan Restaurant
35,Ramamurthy Nagar,3.0,Indian Restaurant,Indie Movie Theater,Optical Shop,Shoe Store,Snack Place


In [99]:
bangalore_merged.loc[bangalore_merged['Cluster Labels'] == 4, bangalore_merged.columns[[0] + list(range(5, bangalore_merged.shape[1]))]]

,Locality,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
118,Neriga,4.0,Resort,Yoga Studio,Dry Cleaner,Food & Drink Shop,Food
